```
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
=================================================================================================================
Project: Handwritten Digits Recognition
File: notebooks/00_quickstart.ipynb
Author: Mobin Yousefi (GitHub: github.com/mobinyousefi-cs)
Created: 2025-11-02
Updated: 2025-11-02
License: MIT License (see LICENSE file for details)
=

Description:
Hands-on quickstart: train, evaluate, predict, and serve the MNIST model.

Usage:
Run the notebook top-to-bottom inside the project virtualenv.

Notes:
- Uses the package from src/ via editable install (pip install -e .).
- Checkpoints are written to artifacts/.

============================================================================
"""
```

# 0) Environment Check
Make sure you're running this **inside the repo root** with the package installed (editable).
If not, run in a terminal:

```bash
python -m venv .venv && source .venv/bin/activate  # Windows: .venv\\Scripts\\activate
pip install -U pip && pip install -e .[dev]
```

In [ ]:
# 1) Imports and sanity checks
from pathlib import Path
import torch
from handwritten_digits import __version__
from handwritten_digits.config import ARTIFACTS_DIR, DEFAULT_DATA_ROOT
from handwritten_digits.model import MnistNet
from handwritten_digits.data import mnist_loaders
from handwritten_digits.utils import device, save_checkpoint
print('Package version:', __version__)
print('Using device:', device())
ARTIFACTS_DIR, DEFAULT_DATA_ROOT

In [ ]:
# 2) Quick Train Loop (1-2 epochs for demo)
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm

train_loader, test_loader = mnist_loaders(str(DEFAULT_DATA_ROOT), batch_size=128)
net = MnistNet().to(device())
opt = optim.Adam(net.parameters(), lr=1e-3)

for epoch in range(1, 2 + 1):
    net.train()
    total = 0
    running = 0.0
    for x, y in tqdm(train_loader, desc=f'Epoch {epoch}/2'):
        x, y = x.to(device()), y.to(device())
        opt.zero_grad()
        logits = net(x)
        loss = F.cross_entropy(logits, y)
        loss.backward()
        opt.step()
        running += loss.item() * x.size(0)
        total += x.size(0)
    print('train_loss=', running/total)

# Save checkpoint
ckpt = ARTIFACTS_DIR / 'model_quickstart.pt'
save_checkpoint({'model': net.state_dict(), 'epoch': epoch}, ckpt)
ckpt

In [ ]:
# 3) Evaluate
import numpy as np
net.eval()
correct = 0
count = 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device()), y.to(device())
        logits = net(x)
        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        count += y.size(0)
acc = correct / count
print(f'Test accuracy: {acc:.2%}')

In [ ]:
# 4) Single Image Prediction Utility (same as CLI logic)
from PIL import Image, ImageOps
import numpy as np

def preprocess_image_pil(img: Image.Image) -> torch.Tensor:
    # MNIST normalization
    if np.array(img).mean() < 127:
        img = ImageOps.invert(img)
    img = ImageOps.pad(img, (28, 28), method=Image.BILINEAR, color=0, centering=(0.5, 0.5))
    arr = np.asarray(img, dtype=np.float32) / 255.0
    arr = (arr - 0.1307) / 0.3081
    return torch.from_numpy(arr).unsqueeze(0).unsqueeze(0)

# Demo with a random canvas digit (optional to load your own image path)
# img = Image.open('path/to/digit.png').convert('L')
# x = preprocess_image_pil(img).to(device())
# with torch.no_grad():
#     p = torch.softmax(net(x), dim=1)
#     print('Pred:', int(p.argmax(1)), 'Conf:', float(p.max()))


## 5) Serving (FastAPI)
From terminal:
```bash
hwr-serve --weights artifacts/model_quickstart.pt --host 0.0.0.0 --port 8000
# Then POST an image to http://localhost:8000/predict
```